# NIRCam CRDS Completeness Check

This notebook captures all supported NIRCam configurations (exp_type/detector/subarray/filter/pupil/etc) and queries CRDS using each configuration, in order to be sure that appropriate reference files exist.

Page listing exp_types and which pipelines they are to be processed through: 
https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/main.html#pipelines-vs-exposure-type

Page listing observing templates and corresponding exposure types:
https://innerspace.stsci.edu/pages/viewpage.action?spaceKey=JSSE&title=Templates+-+modes

MIMF are all FULL and tagged as NRC_IMAGE
What about coronagraphic TA? I skipped over all of these as they will only be processed through calwebb_detector1 and the dark step will be skipped for TA data. All other detector1 reference files have the subarray extracted from the full frame.



## Table of Contents

*Table of Contents:*,
* [Lists of Reference Files to Search For](#reffile_lists)
* [Lists of Allowed Parameter Combinations, by Exposure Type](#allowed_parameters)
    * [Imaging Mode Parameters](#imaging_params)
    * [WFSS Mode Parameters](#wfss_params)
    * [Imaging Time Series Parameters](#tsimage_params)
    * [Grism Time Series Parameters](#tsgrism_params)
    * [Coronagraphy Parameters](#coron_params)
    * [Engineering Imaging Parameters](#engimg_params)
    * [Dark Parameters](#dark_params)
    * [Target Acquisition Parameters](#ta_params)
    * [Target Acquisition Confirmation Parameters](#taconfirm_params)
    * [Focus Parameters](#focus_params)
    * [Flat Parameters](#flat_params)
    * [LED Parameters](#led_params)
    * [Grism Parameters](#grism_params)
* [Function Definitions](#function_definitions)
* [Create All Possible Combinations of Paramters](#create_combinations)
    * [Imaging Mode](#imaging_combos)
    * [WFSS Mode](#wfss_combos)
    * [Imaging Time Series](#tsimage_combos)
    * [Grism Time Series](#tsgrism_combos)
    * [Coronagraphy](#coron_combos)
    * [Engineering Imaging](#engimg_combos)
    * [Dark](#dark_combos)
    * [Target Acquisition](#ta_combos)
    * [Target Acquisition Confirmation](#taconfirm_combos)
    * [Focus](#focus_combos)
    * [Flat](#flat_combos)
    * [LED](#led_combos)
    * [Grism](#grism_combos)
* [Run CRDS Queries](#run_queries)
    * [Imaging Mode Queries](#imaging_queries)
    * [WFSS Mode Queries](#wfss_queries)
    * [Imaging Time Series Queries](#tsimage_queries)
    * [Grism Time Series Queries](#tsgrism_queries)
    * [Coronagraphy Queries](#coron_queries)
    * [Engineering Imaging Queries](#engimg_queries)
    * [Dark Queries](#dark_queries)
    * [Target Acquisition Queries](#ta_queries)
    * [Target Acquisition Confirmation Queries](#taconfirm_queries)
    * [Focus Queries](#focus_queries)
    * [Flat Queries](#flat_queries)
    * [LED Queries](#led_queries)
    * [Grism Queries](#grism_queries)
* [Final Summary](#final_summary)

make sure you have the right exp_types for engineering imaging. It can be several values. right now I think I'm using only nrc_image

In [ ]:
import datetime

import crds
from crds import CrdsLookupError

---
<a id='reffile_lists'></a>
## Lists of reference files we are looking for

In [ ]:
# What about parameter reference files? I guess these aren't necessary for the pipeline to run
# but it could still be useful to know which we have and which we don't

# Looks like we can't just search for all reference files in one go. PSFMASK shows up as
# not found in cases where CORONMSK is not present, as well as in cases where CORONMSK
# is 'N/A' or some other random string.


level1b_reffiles = ['mask', 'ipc', 'saturation', 'superbias',
                    'linearity', 'dark', 'gain', 'readnoise',
                   'persat', 'trapdensity', 'trappars']

level2a_imaging_reffiles = ['distortion', 'area', 'flat', 'photom',
                           ]
level2a_grism_reffiles = ['distortion', 'area', 'flat', 'photom',
                          'specwcs', 'wavelengthrange']

level3_imaging_reffiles = ['distortion']
level3_wfss_reffiles = ['wfssbkg', 'apcorr', 'abvegaoffset']
level3_coron_reffiles = ['psfmask']
level3_tso_reffiles = ['tsophot']

all_reffiles = ['mask', 'ipc', 'saturation', 'superbias',
                'linearity', 'dark', 'gain', 'readnoise',
                'persat', 'trapdensity', 'trappars',
                'distortion', 'area', 'flat', 'photom',
                'specwcs', 'wavelengthrange',
                'wfssbkg', 'apcorr', 'abvegaoffset',
                'psfmask', 'tsophot']

---
<a id='allowed_parameters'></a>
## Lists of allowed parameters, by exposure type

<a id='imaging_params'></a>
### Imaging

NRC_IMAGE data are run through calwebb_detector1, calwebb_image2, and calwebb_image3

In [ ]:
nrc_image_exptype = 'NRC_IMAGE'
nrc_image_sw_dets = ['NRCA1', 'NRCA2', 'NRCA3', 'NRCA4', 'NRCB1', 'NRCB2', 'NRCB3', 'NRCB4']
nrc_image_sw_subarrays = ['FULL', 'SUB640', 'SUB320', 'SUB160']
nrc_image_sw_filterpupil = ['F070W/CLEAR', 'F090W/CLEAR', 'F115W/CLEAR', 'F150W/CLEAR', 'F150W2/CLEAR',
                            'F200W/CLEAR', 'F140M/CLEAR', 'F182M/CLEAR', 'F210M/CLEAR', 'F187N/CLEAR',
                            'F212N/CLEAR', 'WLP4/CLEAR', 'F150W2/F164N']

In [ ]:
nrc_image_lw_dets = ['NRCALONG', 'NRCBLONG']
nrc_image_lw_subarrays = nrc_image_sw_subarrays
nrc_image_lw_filterpupil = ['F277W/CLEAR', 'F322W2/CLEAR', 'F356W/CLEAR', 'F444W/CLEAR',
                            'F250M/CLEAR', 'F300M/CLEAR',  'F335M/CLEAR', 'F360M/CLEAR',
                            'F410M/CLEAR', 'F430M/CLEAR', 'F460M/CLEAR',  'F480M/CLEAR',
                            'F322W2/F323N', 'F444W/F405N', 'F444W/F466N',
                            'F444W/F470N']

In [ ]:
nrc_image_sw_p_dets = ['NRCB1']
nrc_image_sw_p_subarrays = ['SUB400P', 'SUB160P', 'SUB64P']
nrc_image_sw_p_filterpupil = nrc_image_sw_filterpupil

In [ ]:
nrc_image_lw_p_dets = ['NRCBLONG']
nrc_image_lw_p_subarrays = ['SUB400P', 'SUB160P', 'SUB64P']
nrc_image_lw_p_filterpupil = nrc_image_lw_filterpupil

<a id='wfss_params'></a>
### WFSS

WFSS data are run through calwebb_detector1, calwebb_spec2, calwebb_spec3

In [ ]:
nrc_wfss_exptype = 'NRC_WFSS'
nrc_wfss_detectors = ['NRCALONG', 'NRCBLONG']
nrc_wfss_subarrays = ['FULL']
nrc_wfss_lw_filterpupil = ['F277W/GRISMR', 'F322W2/GRISMR', 'F356W/GRISMR', 'F444W/GRISMR',
                           'F250M/GRISMR', 'F300M/GRISMR', 'F335M/GRISMR', 'F360M/GRISMR',
                           'F410M/GRISMR', 'F430M/GRISMR', 'F460M/GRISMR', 'F480M/GRISMR',
                           'F277W/GRISMC', 'F322W2/GRISMC', 'F356W/GRISMC', 'F444W/GRISMC',
                           'F250M/GRISMC', 'F300M/GRISMC', 'F335M/GRISMC', 'F360M/GRISMC',
                           'F410M/GRISMC', 'F430M/GRISMC', 'F460M/GRISMC', 'F480M/GRISMC']

<a id='tsimage_params'></a>
### Imaging Time Series

Imaging Time Series data are run through calwebb_detector1, calwebb_image2, calwebb_tso3

In [ ]:
nrc_tsimg_exptype = 'NRC_TSIMAGE'
nrc_tsimg_sw_detectors = ['NRCB1']
nrc_tsimg_sw_subarrays = ['FULL', 'SUB400P', 'SUB160P', 'SUB64P']
nrc_tsimg_sw_filterpupil = ['F070W/CLEAR', 'F090W/CLEAR', 'F115W/CLEAR', 'F150W/CLEAR',
                            'F150W2/CLEAR', 'F200W/CLEAR', 'F140M/CLEAR',  'F182M/CLEAR',
                            'F210M/CLEAR', 'F187N/CLEAR', 'F212N/CLEAR', 'WLP4/CLEAR',
                            'F150W/WLP8', 'F200W/WLP8', 'F140M/WLP8', 'F182M/WLP8',
                            'F210M/WLP8', 'F187N/WLP8', 'F212N/WLP8']

In [ ]:
nrc_tsimg_lw_detectors = ['NRCBLONG']
nrc_tsimg_lw_subarrays = ['FULL', 'SUB400P', 'SUB160P', 'SUB64P']
nrc_tsimg_lw_filterpupil = ['F277W/CLEAR', 'F322W2/CLEAR', 'F356W/CLEAR', 'F444W/CLEAR',
                            'F250M/CLEAR', 'F300M/CLEAR',  'F335M/CLEAR', 'F360M/CLEAR',
                            'F410M/CLEAR', 'F430M/CLEAR', 'F460M/CLEAR', 'F480M/CLEAR',
                            'F322W2/F323N', 'F322W2/F405N', 'F410M/F405N', 'F444W/F466N',
                            'F444W/F470N']

These combinations are for data that are collected contemporaneously with grism time
series data in the LW channel

In [ ]:
nrc_tsgrism_sw_detectors = ['NRCA1', 'NRCA3']
nrc_tsgrism_sw_subarrays = ['FULL', 'SUBGRISM256', 'SUBGRISM128', 'SUBGRISM64']
nrc_tsgrism_sw_filterpupils = ['F070W/WLP8', 'F140M/WLP8', 'F182M/WLP8', 'F210M/WLP8',
                               'F187N/WLP8', 'F212N/WLP8', 'WLP4/CLEAR']

<a id='tsgrism_params'></a>
### Grism Time Series

Grism Time Series data are run through calwebb_detector1, calwebb_image2, calwebb_tso3

In [ ]:
nrc_tsgrism_exptype = 'NRC_TSGRISM'
nrc_tsgrism_lw_detectors = ['NRCALONG']
nrc_tsgrism_lw_subarrays = ['FULL', 'SUBGRISM256', 'SUBGRISM128', 'SUBGRISM64']
nrc_tsgrism_lw_filterpupil = ['F277W/GRISMR', 'F322W2/GRISMR', 'F356W/GRISMR', 'F444W/GRISMR']

<a id='coron_params'></a>
### Coronagraphy

Coronagraphy data are run through calwebb_detector1, calwebb_image2, calwebb_coron3

In [ ]:
nrc_coron_exptype = 'NRC_CORON'

In [ ]:
nrc_coron_sw_round_detectors = ['NRCA2']
nrc_coron_sw_round_mask = ['MASK210R']
nrc_coron_sw_subarrays = ['FULL', 'SUB640A210R']
nrc_coron_sw_round_filterpupils = ['F200W/MASKRND', 'F182M/MASKRND', 'F210M/MASKRND',
                                   'F187N/MASKRND', 'F212N/MASKRND']

nrc_coron_sw_bar_detectors = ['NRCA4']
nrc_coron_sw_bar_mask = ['FULL', 'MASKSWB']
nrc_coron_sw_bar_subarrays = ['FULL', 'SUB640ASWB']
nrc_coron_sw_bar_filterpupils = ['F200W/MASKBAR', 'F182M/MASKBAR', 'F210M/MASKBAR',
                                   'F187N/MASKBAR', 'F212N/MASKBAR']

In [ ]:
nrc_coron_lw_detectors = ['NRCALONG']
nrc_coron_lw_round_335_mask = ['MASK335R']
nrc_coron_lw_round_335_subarrays = ['FULL', 'SUB320A335R']
nrc_coron_lw_round_335_filterpupils = ['F322W2/MASKRND', 'F356W/MASKRND', 'F250M/MASKRND',
                                   'F300M/MASKRND', 'F335M/MASKRND', 'F360M/MASKRND',
                                   'F410M/MASKRND']

nrc_coron_lw_round_430_mask = ['MASK430R']
nrc_coron_lw_round_430_subarrays = ['FULL', 'SUB320A430R']
nrc_coron_lw_round_430_filterpupils = ['F322W2/MASKRND', 'F356W/MASKRND', 'F444W/MASKRND',
                                       'F250M/MASKRND', 'F300M/MASKRND', 'F335M/MASKRND',
                                       'F360M/MASKRND', 'F410M/MASKRND', 'F430M/MASKRND',
                                       'F460M/MASKRND']

nrc_coron_lw_bar_mask = ['MASKLWB']
nrc_coron_lw_bar_subarrays = ['FULL', 'SUB320ALWB']
nrc_coron_lw_bar_filterpupils = ['F277W/MASKBAR', 'F356W/MASKBAR', 'F444W/MASKBAR',
                                 'F250M/MASKBAR', 'F300M/MASKBAR', 'F335M/MASKBAR',
                                 'F360M/MASKBAR', 'F410M/MASKBAR', 'F430M/MASKBAR',
                                 'F460M/MASKBAR', 'F480M/MASKBAR']

<a id='engimg_params'></a>
### Engineering Imaging

This mode allows just about any filter/pupil/subarray combination. Latest talk is for having engineering imaging data stop after calwebb_detector1, which means no filter-dependent reference files. This makes the check much simpler, as we can just include a single filter/pupil pair in the set of parameters. No need to worry about checking all possible combinations.

In [ ]:
nrc_engimg_sw_detectors = ['NRCB1']
nrc_engimg_sw_subarrays = ['FULL', 'SUB400P', 'SUB160P', 'SUB64P']
nrc_engimg_sw_filterpupils = ['F200W/CLEAR']

In [ ]:
nrc_engimg_lw_detectors = ['NRCBLONG']
nrc_engimg_lw_subarrays = ['FULL', 'SUB400P', 'SUB160P', 'SUB64P']
nrc_engimg_lw_filterpupils = ['F444W/CLEAR']

In [ ]:
nrc_engimg_sw_round_detectors = ['NRCB1']
nrc_engimg_sw_round_mask = ['MASK210R']
nrc_engimg_sw_round_subarrays = ['FULL', 'SUB640B210R']
nrc_engimg_sw_round_filterpupils = ['F200W/MASKRND', 'F182M/MASKRND', 'F210M/MASKRND',
                                   'F187N/MASKRND', 'F212N/MASKRND']

nrc_engimg_sw_bar_detectors = ['NRCB3']
nrc_engimg_sw_bar_mask = ['FULL', 'MASKSWB']
nrc_engimg_sw_bar_subarrays = ['FULL', 'SUB640BSWB']
nrc_engimg_sw_bar_filterpupils = ['F200W/MASKBAR', 'F182M/MASKBAR', 'F210M/MASKBAR',
                                   'F187N/MASKBAR', 'F212N/MASKBAR']

In [ ]:
nrc_engimg_lw_detectors = ['NRCBLONG']
nrc_engimg_lw_round_335_mask = ['MASK335R']
nrc_engimg_lw_round_335_subarrays = ['FULL', 'SUB320B335R']
nrc_engimg_lw_round_335_filterpupils = ['F322W2/MASKRND', 'F356W/MASKRND', 'F250M/MASKRND',
                                   'F300M/MASKRND', 'F335M/MASKRND', 'F360M/MASKRND',
                                   'F410M/MASKRND']

nrc_engimg_lw_round_430_mask = ['MASK430R']
nrc_engimg_lw_round_430_subarrays = ['FULL', 'SUB320B430R']
nrc_engimg_lw_round_430_filterpupils = ['F322W2/MASKRND', 'F356W/MASKRND', 'F444W/MASKRND',
                                       'F250M/MASKRND', 'F300M/MASKRND', 'F335M/MASKRND',
                                       'F360M/MASKRND', 'F410M/MASKRND', 'F430M/MASKRND',
                                       'F460M/MASKRND']

nrc_engimg_lw_bar_mask = ['MASKLWB']
nrc_engimg_lw_bar_subarrays = ['FULL', 'SUB320BLWB']
nrc_engimg_lw_bar_filterpupils = ['F277W/MASKBAR', 'F356W/MASKBAR', 'F444W/MASKBAR',
                                 'F250M/MASKBAR', 'F300M/MASKBAR', 'F335M/MASKBAR',
                                 'F360M/MASKBAR', 'F410M/MASKBAR', 'F430M/MASKBAR',
                                 'F460M/MASKBAR', 'F480M/MASKBAR']


In [ ]:
nrc_engimg_fp_a3_detectors = ['A3']
nrc_engimg_fp_a3_subarrays = ['SUB64FP1A', 'SUB64FP1A']

nrc_engimg_fp_b4_detectors = ['B4']
nrc_engimg_fp_b4_subarrays = ['SUB64FP1B', 'SUB8FP1B']

In [ ]:
nrc_engimg_dhs_a_detectors = ['NRCA3']
nrc_engimg_dhs_a_subarrays = ['SUB96DHSPILA']
nrc_engimg_dhs_filterpupil = ['F115W/GDHS0', 'F150W/GDHS0', 'F150W2/GDHS0',
                              'F200W/GDHS0', 'F140M/GDHS0', 'F182M/GDHS0', 'F210M/GDHS0', 'F187N/GDHS0',
                              'F212N/GDHS0', 'F115W/GDHS60', 'F150W/GDHS60', 'F150W2/GDHS60',
                              'F200W/GDHS60', 'F140M/GDHS60', 'F182M/GDHS60', 'F210M/GDHS60', 'F187N/GDHS60',
                              'F212N/GDHS60']

nrc_engimg_dhs_b_detectors = ['NRCB4']
nrc_engimg_dhs_b_subarrays = ['SUB96DHSPILB']

<a id='dark_params'></a>
### Darks

Dark data are run through calwebb_dark

In [ ]:
nrc_dark_exptype = 'NRC_DARK'
nrc_dark_sw_dets = ['NRCA1', 'NRCA2', 'NRCA3', 'NRCA4', 'NRCB1', 'NRCB2', 'NRCB3', 'NRCB4']
nrc_dark_sw_subarrays = ['FULL', 'SUB640', 'SUB320', 'SUB160']

# Only need one combination here since detector1 reffiles are not
# filter dependent
nrc_dark_sw_pupils = ['F090W/FLAT']

In [ ]:
nrc_dark_lw_dets = ['NRCALONG', 'NRCBLONG']
nrc_dark_lw_subarrays = ['FULL', 'SUB640', 'SUB320', 'SUB160']

# Only need one combination here since detector1 reffiles are not
# filter dependent
nrc_dark_lw_pupils = ['F444W/FLAT']

In [ ]:
nrc_dark_sw_p_dets = ['NRCB1']
nrc_dark_p_subarrays = ['SUB400P', 'SUB160P', 'SUB64P']

In [ ]:
nrc_dark_lw_p_dets = ['NRCBLONG']

In [ ]:
# Include grism time series subarrays
# (imaging time series subarrays are included within imaging mode above)
nrc_dark_grismts_sw_dets = nrc_tsgrism_sw_detectors
nrc_dark_grismts_sw_subarrays = nrc_tsgrism_sw_subarrays

nrc_dark_grismts_lw_dets = nrc_tsgrism_lw_detectors
nrc_dark_grismts_lw_subarrays = nrc_tsgrism_lw_subarrays

In [ ]:
# Coronagraphy subarrays...are not covered? is sub640 the same as sub640 in imaging mode?

In [ ]:
# TA observations will have dark current subtraction skipped, so we
# need to omit the dark search for them

In [ ]:
# subarrays shouldn't matter much here since we are doing only calwebb_detector1
# reffiles. Of those, only the dark is not extracted from the full frame, and here, for nrc_dark
# data, we will not be doing dark subtraction.

<a id='ta_params'></a>
### Target Acquisition

Target Acquisition data are run through calwebb_detector1 and calwebb_image2

In [ ]:
nrc_ta_exptype = 'NRC_TACQ'

#### Coronagraphy

In [ ]:
nrc_coron_ta_subarrays = ['SUB128', 'SUB64']

nrc_coron_ta_round_detectors = ['NRCA2']
nrc_coron_ta_round_mask = ['MASK210R']
nrc_coron_ta_sw_round_filterpupils = ['F210M/MASKRND']

nrc_coron_ta_bar_detectors = ['NRCA4']
nrc_coron_ta_bar_mask = ['MASKSWB']
nrc_coron_ta_sw_bar_filterpupils = ['F210M/MASKBAR']

In [ ]:
nrc_coron_ta_lw_detectors = ['NRCALONG']
nrc_coron_ta_round_335_mask = ['MASK335R']
nrc_coron_ta_lw_round_335_filterpupils = ['F335M/MASKRND']

nrc_coron_ta_round_430_mask = ['MASK430R']
nrc_coron_ta_lw_round_430_filterpupils = ['F335M/MASKRND']

nrc_coron_ta_lw_bar_mask = ['MASKLWB']
nrc_coron_ta_lw_bar_filterpupils = ['F335M/MASKBAR']

#### Imaging Time Series

In [ ]:
nrc_tsimg_ta_detectors = ['NRCBLONG']
nrc_tsimg_ta_subarrays = ['SUB32TATS']
nrc_tsimg_ta_filterpupils = ['F335M/CLEAR', 'F444W/F405N']

#### Grism Time Series

In [ ]:
nrc_tsgrism_ta_detectors = ['NRCALONG']
nrc_tsgrism_ta_subarrays = ['SUB32TATSGRISM']
nrc_tsgrism_ta_filterpupils = ['F335M/CLEAR', 'F444W/F405N']

<a id='taconf_params'></a>
### Target Acquisition Confirmation

These parameters match the coronagraphy TA parameters above.
Target Acquisition Confirmation data will be run though calwebb_detector1 and calwebb_image2

In [ ]:
nrc_ta_conf_exptype = 'NRC_TACONF'

In [ ]:
nrc_ta_conf_subarrays = ['SUB128', 'SUB64']

nrc_ta_conf_sw_round_detectors = ['NRCA2']
nrc_ta_conf_sw_round_mask = ['MASK210R']
nrc_ta_conf_sw_round_filterpupils = ['F210M/MASKRND']

nrc_ta_conf_sw_bar_detectors = ['NRCA4']
nrc_ta_conf_sw_bar_mask = ['MASKSWB']
nrc_ta_conf_sw_bar_filterpupils = ['F210M/MASKBAR']

In [ ]:
nrc_ta_conf_lw_detectors = ['NRCALONG']
nrc_ta_conf_lw_round_335_mask = ['MASK335R']
nrc_ta_conf_lw_round_335_filterpupils = ['F335M/MASKRND']

nrc_ta_conf_lw_round_430_mask = ['MASK430R']
nrc_ta_conf_lw_round_430_filterpupils = ['F335M/MASKRND']

nrc_ta_conf_lw_bar_mask = ['MASKLWB']
nrc_ta_conf_lw_bar_filterpupils = ['F335M/MASKBAR']

<a id='focus_params'></a>
### Focus

It appears that NRC_FOCUS data can be taken with any combination of filter/pupil (at least, they are all selectable in APT)

These data will be processed through detector1 and image2 pipelines.

In [ ]:
nrc_focus_exptype = 'NRC_FOCUS'
nrc_focus_sw_dets = ['NRCA1', 'NRCA2', 'NRCA3', 'NRCA4', 'NRCB1', 'NRCB2', 'NRCB3', 'NRCB4']
nrc_focus_sw_subarrays = ['FULL']
nrc_focus_sw_filterpupil = ['F070W/CLEAR', 'F090W/CLEAR', 'F115W/CLEAR', 'F150W/CLEAR', 'F150W2/CLEAR',
                            'F200W/CLEAR', 'F140M/CLEAR', 'F182M/CLEAR', 'F210M/CLEAR', 'F187N/CLEAR',
                            'F212N/CLEAR', 'WLP4/CLEAR', 'F150W2/F164N', 'F150W/F164N', 'F150W2/F162M',
                            
                            'F070W/WLP8', 'F090W/WLP8', 'F115W/WLP8', 'F150W/WLP8', 'F150W2/WLP8',
                            'F200W/WLP8', 'F140M/WLP8', 'F182M/WLP8', 'F210M/WLP8', 'F187N/WLP8',
                            'F212N/WLP8', 'WLP4/WLP8',
                            
                            'F070W/WLM8', 'F090W/WLM8', 'F115W/WLM8', 'F150W/WLM8', 'F150W2/WLM8',
                            'F200W/WLM8', 'F140M/WLM8', 'F182M/WLM8', 'F210M/WLM8', 'F187N/WLM8',
                            'F212N/WLM8', 'WLP4/WLM8',
                            
                            'F115W/GDHS0', 'F150W/GDHS0', 'F150W2/GDHS0',
                            'F200W/GDHS0', 'F140M/GDHS0', 'F182M/GDHS0', 'F210M/GDHS0', 'F187N/GDHS0',
                            'F212N/GDHS0', 'F115W/GDHS60', 'F150W/GDHS60', 'F150W2/GDHS60',
                            'F200W/GDHS60', 'F140M/GDHS60', 'F182M/GDHS60', 'F210M/GDHS60', 'F187N/GDHS60',
                            'F212N/GDHS60',
                           
                            'F070W/PINHOLES', 'F090W/PINHOLES', 'F115W/PINHOLES', 'F150W/PINHOLES',
                            'F150W2/PINHOLES', 'F200W/WLM8', 'F140M/PINHOLES', 'F182M/PINHOLES',
                            'F210M/PINHOLES', 'F187N/PINHOLES', 'F212N/WLM8', 'WLP4/PINHOLES',
                            
                            'F070W/MASKIPR', 'F090W/MASKIPR', 'F115W/MASKIPR', 'F150W/MASKIPR',
                            'F150W2/MASKIPR', 'F200W/MASKIPR', 'F140M/MASKIPR', 'F182M/MASKIPR',
                            'F210M/MASKIPR', 'F187N/MASKIPR', 'F212N/MASKIPR', 'WLP4/MASKIPR',
                            
                            'F200W/MASKRND', 'F182M/MASKRND', 'F210M/MASKRND', 'F187N/MASKRND',
                            'F212N/MASKRND',
                            
                            'F200W/MASKBAR', 'F182M/MASKBAR', 'F210M/MASKBAR', 'F187N/MASKBAR',
                            'F212N/MASKBAR']

In [ ]:
nrc_focus_lw_dets = ['NRCALONG', 'NRCBLONG']
nrc_focus_lw_subarrays = ['FULL']
nrc_focus_lw_filterpupil = ['F277W/CLEAR', 'F322W2/CLEAR', 'F356W/CLEAR', 'F444W/CLEAR',
                            'F250M/CLEAR', 'F300M/CLEAR',  'F335M/CLEAR', 'F360M/CLEAR',
                            'F410M/CLEAR', 'F430M/CLEAR', 'F460M/CLEAR',  'F480M/CLEAR',
                            'F322W2/F323N', 'F322W2/F405N', 'F410M/F405N', 'F444W/F466N',
                            'F444W/F470N',
                            
                            'F335M/F323N', 'F356W/F323N',
                            
                            'F410M/F405N',
                            'F460M/F466N', 'F480M/F466N',
                            'F480M/F470N',
                            
                            'F322W2/MASKRND', 'F356W/MASKRND', 'F444W/MASKRND',
                            'F250M/MASKRND', 'F300M/MASKRND', 'F335M/MASKRND',
                            'F360M/MASKRND', 'F410M/MASKRND', 'F430M/MASKRND',
                            'F460M/MASKRND',
                            
                            'F277W/MASKBAR', 'F356W/MASKBAR', 'F444W/MASKBAR',
                            'F250M/MASKBAR', 'F300M/MASKBAR', 'F335M/MASKBAR',
                            'F360M/MASKBAR', 'F410M/MASKBAR', 'F430M/MASKBAR',
                            'F460M/MASKBAR', 'F480M/MASKBAR',
                            
                            'F277W/MASKIPR', 'F322W2/MASKIPR', 'F356W/MASKIPR', 'F444W/MASKIPR',
                            'F250M/MASKIPR', 'F300M/MASKIPR',  'F335M/MASKIPR', 'F360M/MASKIPR',
                            'F410M/MASKIPR', 'F430M/MASKIPR', 'F460M/MASKIPR',  'F480M/MASKIPR',
                            
                            'F277W/GRISMR', 'F322W2/GRISMR', 'F356W/GRISMR', 'F444W/GRISMR',
                            'F250M/GRISMR', 'F300M/GRISMR',  'F335M/GRISMR', 'F360M/GRISMR',
                            'F410M/GRISMR', 'F430M/GRISMR', 'F460M/GRISMR',  'F480M/GRISMR',
                            
                            'F277W/GRISMC', 'F322W2/GRISMC', 'F356W/GRISMC', 'F444W/GRISMC',
                            'F250M/GRISMC', 'F300M/GRISMC',  'F335M/GRISMC', 'F360M/GRISMC',
                            'F410M/GRISMC', 'F430M/GRISMC', 'F460M/GRISMC',  'F480M/GRISMC',
                            
                            'F277W/PINHOLES', 'F322W2/PINHOLES', 'F356W/PINHOLES', 'F444W/PINHOLES',
                            'F250M/PINHOLES', 'F300M/PINHOLES',  'F335M/PINHOLES', 'F360M/PINHOLES',
                            'F410M/PINHOLES', 'F430M/PINHOLES', 'F460M/PINHOLES',  'F480M/PINHOLES'
                           ]

<a id='flat_params'></a>
### Flat

NRC_FLAT data will be processed through calwebb_detector1

In [ ]:
nrc_flat_exp_type = 'NRC_FLAT'
nrc_flat_sw_a_detectors = ['NRCA1', 'NRCA2', 'NRCA3', 'NRCA4']
nrc_flat_sw_a_subarrays = ['FULL']

# Since these data will only be run through calwebb_detector1, there are no filter-dependent
# reference files to search for
nrc_flat_sw_a_filterpupils = ['F070W/CLEAR']

In [ ]:
nrc_flat_sw_b_detectors = ['NRCB1', 'NRCB2', 'NRCB3', 'NRCB4']
nrc_flat_sw_b_subarrays = ['FULL', 'SUB640', 'SUB320', 'SUB160', 'SUB400P', 'SUB160P', 'SUB64P']

# Since these data will only be run through calwebb_detector1, there are no filter-dependent
# reference files to search for
nrc_flat_sw_b_filterpupils = ['F070W/CLEAR']

In [ ]:
nrc_flat_lw_a_detectors = ['NRCALONG']
nrc_flat_lw_a_subarrays = ['FULL']

# Since these data will only be run through calwebb_detector1, there are no filter-dependent
# reference files to search for
nrc_flat_lw_a_filterpupils = ['F444W/CLEAR']

In [ ]:
nrc_flat_lw_b_detectors = ['NRCBLONG']
nrc_flat_lw_b_subarrays = ['FULL', 'SUB640', 'SUB320', 'SUB160', 'SUB400P', 'SUB160P', 'SUB64P']

# Since these data will only be run through calwebb_detector1, there are no filter-dependent
# reference files to search for
nrc_flat_lw_b_filterpupils = ['F444W/CLEAR']

<a id='led_params'></a>
### LED

NRC_LED data will be processed through calwebb_detector1, so no need to worry about filter/pupil combinations

In [ ]:
nrc_led_exp_type = 'NRC_LED'
nrc_led_sw_detectors = ['NRCA1', 'NRCA2', 'NRCA3', 'NRCA4', 'NRCB1', 'NRCB2', 'NRCB3', 'NRCB4']
nrc_led_sw_subarrays = ['FULL']
nrc_led_sw_filterpupils = ['F070W/CLEAR']

In [ ]:
nrc_led_lw_detectors = ['NRCALONG', 'NRCBLONG']
nrc_led_lw_subarrays = ['FULL']
nrc_led_lw_filterpupils = ['F444W/CLEAR']

<a id='grism_params'></a>
### NRC_GRISM

NRC_GRISM data will be processed through calwebb_detector1 - This exp_type should only come about when using engineering imaging with the grism in place.

In [ ]:
nrc_engimg_lw_grism_exp_type = 'NRC_GRISM'
nrc_engimg_lw_grism_detectors = ['NRCALONG', 'NRCBLONG']
nrc_engimg_lw_grism_subarrays = ['FULL', 'SUB640', 'SUB320', 'SUB160']
nrc_engimg_lw_grism_filterpupils = ['F277W/GRISMR', 'F322W2/GRISMR', 'F356W/GRISMR', 'F444W/GRISMR',
                                    'F250M/GRISMR', 'F300M/GRISMR', 'F335M/GRISMR', 'F360M/GRISMR',
                                    'F410M/GRISMR', 'F430M/GRISMR', 'F460M/GRISMR', 'F480M/GRISMR',
                                    'F277W/GRISMC', 'F322W2/GRISMC', 'F356W/GRISMC', 'F444W/GRISMC',
                                    'F250M/GRISMC', 'F300M/GRISMC', 'F335M/GRISMC', 'F360M/GRISMC',
                                    'F410M/GRISMC', 'F430M/GRISMC', 'F460M/GRISMC', 'F480M/GRISMC']

In [ ]:
nrc_engimg_lw_p_grism_detectors = ['NRCBLONG']
nrc_engimg_lw_p_grism_subarrays = ['SUB400P', 'SUB160P', 'SUB64P']

---
<a id='function_definitions'></a>
## Function definitions

In [ ]:
def batch_query(config_list, reffiles):
    """Wrapper around query_crds so that it can be called on a list of
    configuration dictionaries.
    
    Parameters
    ----------
    config_list : list
        List of instrument configuration dictionaries
        
    reffiles : list
        List of reference file types to search for
        
    Returns
    -------
    missing_files : dict
        Dictionary of missing reference files. Keys are backslash-separated lists
        of instrument configurations. Values are lists of the missing file types
    """
    missing_files = {}
    missing = False
    for entry in config_list:
        query = query_crds(entry, reffiles)
        if 'MISSING' in query.values():
            missing = True
            missing_types = [filetype for filetype, val in query.items() if val == 'MISSING']
            key = dict_keys_to_string(entry)
            missing_files[key] = missing_types
            #for missed in missing_types:
            #    print('{} missing for: {}'.format(missed, entry))

    if not missing:
        print('No missing reference files.')
        
    if missing:
        print("Some reference files are missing!")
    
    return missing_files

In [ ]:
def create_parameter_dict(exp_type, detector, subarray, filter_name, pupil_name,
                          coronmask_name=None, tso_visit=False):
    """Create a dictionary to be used as input to the CRDS getreferences
    function 
    
    Parameters
    ----------
    exp_type : str
        Exposure type (e.g. NRC_IMAGE, NRC_TSGRISM)
        
    detector : str
        e.g. 'NRCA1', 'NRCBLONG'
        
    subarray : str
        Subarray name (e.g. 'FULL', 'SUBGRISM128')
        
    filter_name : str
        Filter. (e.g. 'F444W')
        
    pupil_name : str
        Pupil. (e.g. 'CLEAR')
        
    coronmask_name : str
        Coronagraphic mask (e.g. MASKBAR)
        
    tso_visit : bool
        Whether or not the case to search for is a TSO visit

    Returns
    -------
    crds_dict : dict
        Dictionary of information necessary to select refernce files
        via getreferences().
    """
    crds_dict = {}
    crds_dict['INSTRUME'] = 'NIRCAM'
    crds_dict['READPATT'] = 'RAPID'
    crds_dict['EXP_TYPE'] = exp_type.upper()
    crds_dict['DETECTOR'] = detector.upper()
    crds_dict['SUBARRAY'] = subarray.upper()
    crds_dict['FILTER'] = filter_name.upper()
    crds_dict['PUPIL'] = pupil_name.upper()
    crds_dict['TSOVISIT'] = tso_visit
    
    if coronmask_name is not None:
        crds_dict['CORONMSK'] = coronmask_name.upper()
    
    if crds_dict['DETECTOR'] in ['NRCALONG', 'NRCBLONG']:
        crds_dict['CHANNEL'] = 'LONG'
    else:
        crds_dict['CHANNEL'] = 'SHORT'

    # Use the current date and time in order to get the most recent
    # reference file
    crds_dict['DATE-OBS'] = datetime.date.today().isoformat()
    current_date = datetime.datetime.now()
    crds_dict['TIME-OBS'] = current_date.time().isoformat()

    return crds_dict

In [ ]:
def dict_keys_to_string(config):
    """Given a dictionary, create a string composed of all the
    keys in the dictionary, separated by back slashes
    
    Parameters
    ----------
    config : dict
        Dictionary of instrument configuration info
        
    Returns
    -------
    combined_str : str
        String composed of all dictionary keys
    """
    combined_str = ''
    for i, key in enumerate(config): 
        if i < (len(config)-1): 
            combined_str = combined_str + '{}/'.format(config[key]) 
        else: 
            combined_str = combined_str + config[key] 
    return combined_str

In [ ]:
def list_of_dictionaries(exposure_type, detectors, subarrays, filterpupils,
                         coronmasks=[], tso_visit=False, detector1=False):
    """Given lists of detectors, subarrays, filters, etc, generate a list of
    parameter dictionaries, each of which can be used to query CRDS. All
    combinations of the entered lists will be added to the list of dictionaries,
    unless the detector1 keyword is True. In that case, Filter/Pupil will be
    ignored since none of the calwebb_detector1 reference files depend on
    filter/pupil.
    
    Parameters
    ----------
    exposure_type : str
        e.g. NRC_IMAGE
        
    detectors : list
        List of detectors e.g. ['NRCB1', 'NRCB2']
        
    subarrays : list
        List of subarrays e.g. ['FULL', 'SUB320']
        
    filterpupils : list
        List of filter/pupil strings e.g. ['F444W/CLEAR', 'F356W/CLEAR']
    
    coronmasks : list
        Optional list of coronagraphic masks e.g. ['MASKBAR']
        
    tso_visit : bool
        Whether or not the case to be searched is a TSO visit
        
    detector1 : bool
        If True, the dictionaries will be used for querying for calwebb_detector1
        reference files, which do not depend on filter. One filter and pupil
        value will be used rather than all combinations with all filters.
        
    Returns
    -------
    query_dicts : list
        List of CRDS-formatted dcitionaries
    """
    query_dicts = []
    for detector in detectors:
        for subarray in subarrays:
            if detector1:
                filter_name, pupil_name = filterpupils[0].split('/')
                query_dicts.append(create_parameter_dict(exposure_type, detector, 
                                                         subarray, filter_name, pupil_name,
                                                         tso_visit=tso_visit))                        
            else:
                for filterpupil in filterpupils:
                    filter_name, pupil_name = filterpupil.split('/')
                    if len(coronmasks) == 0:
                        query_dicts.append(create_parameter_dict(exposure_type, detector, 
                                                                 subarray, filter_name, pupil_name,
                                                                 tso_visit=tso_visit))
                    else:
                        for coronmask in coronmasks:
                            query_dicts.append(create_parameter_dict(exposure_type, detector, 
                                                                     subarray, filter_name,
                                                                     pupil_name,
                                                                     coronmask_name=coronmask,
                                                                     tso_visit=tso_visit))
    return query_dicts

In [ ]:
def query_crds(parameter_dict, reffile_types):
    """Query CRDS for a given set of reference files with a given set of instrument parameters
    
    Parameters
    ----------
    parameter_dict : dict
        Dictionary of instrument parameters to use in the query. Output of 
        create_parameter_dict().
        
    reffile_types : list
        List of reference file types to look for.
    """
    try:
        reffile_mapping = crds.getrecommendations(parameter_dict, reftypes=reffile_types)
    except CrdsLookupError as e:
        #raise ValueError(("ERROR: CRDSLookupError when trying to find reference "
        #                  "files for parameters: {}".format(parameter_dict)))
        pass
        
    # Check for missing files
    results = {}
    for key, value in reffile_mapping.items():
        if "NOT FOUND No match found." in value:
            results[key] = 'MISSING'
        else:
            results[key] = reffile_mapping[key]
    return results

---
<a id='create_combinations'></a>
## Create parameter dictionaries

<a id='imaging_combos'></a>
### Imaging

In [ ]:
nrc_image_sw_lvl1 = list_of_dictionaries(nrc_image_exptype, nrc_image_sw_dets,
                                         nrc_image_sw_subarrays, nrc_image_sw_filterpupil,
                                         detector1=True)
nrc_image_lw_lvl1 = list_of_dictionaries(nrc_image_exptype, nrc_image_lw_dets,
                                         nrc_image_lw_subarrays, nrc_image_lw_filterpupil,
                                         detector1=True)
nrc_image_sw_p_lvl1 = list_of_dictionaries(nrc_image_exptype, nrc_image_sw_p_dets,
                                           nrc_image_sw_p_subarrays, nrc_image_sw_p_filterpupil,
                                           detector1=True)
nrc_image_lw_p_lvl1 = list_of_dictionaries(nrc_image_exptype, nrc_image_lw_p_dets,
                                           nrc_image_lw_p_subarrays, nrc_image_lw_p_filterpupil,
                                           detector1=True)
nrc_image_lvl1 = nrc_image_sw_lvl1 + nrc_image_lw_lvl1 + nrc_image_sw_p_lvl1 + nrc_image_lw_p_lvl1

In [ ]:
nrc_image_sw_lvl2 = list_of_dictionaries(nrc_image_exptype, nrc_image_sw_dets,
                                         nrc_image_sw_subarrays, nrc_image_sw_filterpupil,
                                         detector1=False)
nrc_image_lw_lvl2 = list_of_dictionaries(nrc_image_exptype, nrc_image_lw_dets,
                                         nrc_image_lw_subarrays, nrc_image_lw_filterpupil,
                                         detector1=False)
nrc_image_sw_p_lvl2 = list_of_dictionaries(nrc_image_exptype, nrc_image_sw_p_dets,
                                           nrc_image_sw_p_subarrays, nrc_image_sw_p_filterpupil,
                                           detector1=False)
nrc_image_lw_p_lvl2 = list_of_dictionaries(nrc_image_exptype, nrc_image_lw_p_dets,
                                           nrc_image_lw_p_subarrays, nrc_image_lw_p_filterpupil,
                                           detector1=False)
nrc_image_lvl2 = nrc_image_sw_lvl2 + nrc_image_lw_lvl2 + nrc_image_sw_p_lvl2 + nrc_image_lw_p_lvl2

In [ ]:
print('Number of calwebb_detector1 checks: ', len(nrc_image_lvl1))
print('Number of image2/3 checks: ', len(nrc_image_lvl2))

<a id='wfss_combos'></a>
### WFSS

In [ ]:
nrc_wfss_lvl1 = list_of_dictionaries(nrc_wfss_exptype, nrc_wfss_detectors,
                                         nrc_wfss_subarrays, nrc_wfss_lw_filterpupil,
                                         detector1=True)
nrc_wfss_lvl2 = list_of_dictionaries(nrc_wfss_exptype, nrc_wfss_detectors,
                                         nrc_wfss_subarrays, nrc_wfss_lw_filterpupil,
                                         detector1=False)

<a id='tsimage_combos'></a>
### Imaging Time Series

In [ ]:
nrc_tsimg_sw_lvl1 = list_of_dictionaries(nrc_tsimg_exptype, nrc_tsimg_sw_detectors,
                                         nrc_tsimg_sw_subarrays, nrc_tsimg_sw_filterpupil,
                                         detector1=True)
nrc_tsimg_lw_lvl1 = list_of_dictionaries(nrc_tsimg_exptype, nrc_tsimg_lw_detectors,
                                         nrc_tsimg_lw_subarrays, nrc_tsimg_lw_filterpupil,
                                         detector1=True)
nrc_tsgrism_sw_lvl1 = list_of_dictionaries(nrc_tsimg_exptype, nrc_tsgrism_sw_detectors,
                                           nrc_tsgrism_sw_subarrays, nrc_tsgrism_sw_filterpupils,
                                           detector1=True)

In [ ]:
nrc_tsimg_sw_lvl2 = list_of_dictionaries(nrc_tsimg_exptype, nrc_tsimg_sw_detectors,
                                         nrc_tsimg_sw_subarrays, nrc_tsimg_sw_filterpupil,
                                         detector1=False)
nrc_tsimg_lw_lvl2 = list_of_dictionaries(nrc_tsimg_exptype, nrc_tsimg_lw_detectors,
                                         nrc_tsimg_lw_subarrays, nrc_tsimg_lw_filterpupil,
                                         detector1=False)
nrc_tsgrism_sw_lvl2 = list_of_dictionaries(nrc_tsimg_exptype, nrc_tsgrism_sw_detectors,
                                           nrc_tsgrism_sw_subarrays, nrc_tsgrism_sw_filterpupils,
                                           detector1=False)

<a id='tsgrism_combos'></a>
### Grism Time Series

In [ ]:
nrc_tsgrism_lw_lvl1 = list_of_dictionaries(nrc_tsgrism_exptype, nrc_tsgrism_lw_detectors,
                                           nrc_tsgrism_lw_subarrays, nrc_tsgrism_lw_filterpupil,
                                           detector1=True)
nrc_tsgrism_lw_lvl2 = list_of_dictionaries(nrc_tsgrism_exptype, nrc_tsgrism_lw_detectors,
                                           nrc_tsgrism_lw_subarrays, nrc_tsgrism_lw_filterpupil,
                                           detector1=False)

<a id='coron_combos'></a>
### Coronagraphy

In [ ]:
nrc_coron_sw_round_lvl1 = list_of_dictionaries(nrc_coron_exptype, nrc_coron_sw_round_detectors,
                                               nrc_coron_sw_subarrays, nrc_coron_sw_round_filterpupils,
                                               detector1=True)
nrc_coron_sw_bar_lvl1 = list_of_dictionaries(nrc_coron_exptype, nrc_coron_sw_bar_detectors,
                                             nrc_coron_sw_subarrays, nrc_coron_sw_bar_filterpupils,
                                             detector1=True)
nrc_coron_lw_round_335_lvl1 = list_of_dictionaries(nrc_coron_exptype, nrc_coron_lw_detectors,
                                                   nrc_coron_lw_round_335_subarrays,
                                                   nrc_coron_lw_round_335_filterpupils,
                                                   detector1=True)
nrc_coron_lw_round_430_lvl1 = list_of_dictionaries(nrc_coron_exptype, nrc_coron_lw_detectors,
                                                   nrc_coron_lw_round_430_subarrays,
                                                   nrc_coron_lw_round_430_filterpupils,
                                                   detector1=True)
nrc_coron_lw_bar_lvl1 = list_of_dictionaries(nrc_coron_exptype, nrc_coron_lw_detectors,
                                                   nrc_coron_lw_bar_subarrays,
                                                   nrc_coron_lw_bar_filterpupils,
                                                   detector1=True)

In [ ]:
nrc_coron_sw_round_lvl2 = list_of_dictionaries(nrc_coron_exptype, nrc_coron_sw_round_detectors,
                                               nrc_coron_sw_subarrays, nrc_coron_sw_round_filterpupils,
                                               coronmasks=nrc_coron_sw_round_mask,
                                               detector1=False)
nrc_coron_sw_bar_lvl2 = list_of_dictionaries(nrc_coron_exptype, nrc_coron_sw_bar_detectors,
                                             nrc_coron_sw_subarrays, nrc_coron_sw_bar_filterpupils,
                                             coronmasks=nrc_coron_sw_bar_mask,
                                             detector1=False)
nrc_coron_lw_round_335_lvl2 = list_of_dictionaries(nrc_coron_exptype, nrc_coron_lw_detectors,
                                                   nrc_coron_lw_round_335_subarrays,
                                                   nrc_coron_lw_round_335_filterpupils,
                                                   coronmasks=nrc_coron_lw_round_335_mask,
                                                   detector1=False)
nrc_coron_lw_round_430_lvl2 = list_of_dictionaries(nrc_coron_exptype, nrc_coron_lw_detectors,
                                                   nrc_coron_lw_round_430_subarrays,
                                                   nrc_coron_lw_round_430_filterpupils,
                                                   coronmasks=nrc_coron_lw_round_430_mask,
                                                   detector1=False)
nrc_coron_lw_bar_lvl2 = list_of_dictionaries(nrc_coron_exptype, nrc_coron_lw_detectors,
                                             nrc_coron_lw_bar_subarrays,
                                             nrc_coron_lw_bar_filterpupils,
                                             coronmasks=nrc_coron_lw_bar_mask,   
                                             detector1=False)

<a id='engimg_combos'></a>
### Engineering Imaging

Engineering imaging data are only processed through calwebb_detector1. Since none of the reference files used in there depend on filters or coronagraphic masks, we don't need to specify those here.

In [ ]:
nrc_engimg_sw_lvl1 = list_of_dictionaries('NRC_IMAGE', nrc_engimg_sw_detectors ,
                                          nrc_engimg_sw_subarrays, nrc_engimg_sw_filterpupils,
                                          detector1=True)
nrc_engimg_lw_lvl1 = list_of_dictionaries('NRC_IMAGE', nrc_engimg_lw_detectors ,
                                          nrc_engimg_lw_subarrays, nrc_engimg_lw_filterpupils,
                                          detector1=True)

In [ ]:
nrc_engimg_sw_coro_round_lvl1 = list_of_dictionaries('NRC_IMAGE', nrc_engimg_sw_round_detectors,
                                                     nrc_engimg_sw_round_subarrays,
                                                     nrc_engimg_sw_round_filterpupils,
                                                     #coronmasks=nrc_engimg_sw_round_mask,
                                                     detector1=True)
nrc_engimg_sw_coro_bar_lvl1 = list_of_dictionaries('NRC_IMAGE', nrc_engimg_sw_bar_detectors,
                                               nrc_engimg_sw_bar_subarrays,
                                               nrc_engimg_sw_bar_filterpupils,
                                               #coronmasks=nrc_engimg_sw_bar_mask,
                                               detector1=True)

In [ ]:
nrc_engimg_lw_coro_round_335_lvl1 = list_of_dictionaries('NRC_IMAGE', nrc_engimg_lw_detectors,
                                                         nrc_engimg_lw_round_335_subarrays,
                                                         nrc_engimg_lw_round_335_filterpupils,
                                                         #coronmasks=nrc_engimg_lw_round_335_mask,
                                                         detector1=True)
nrc_engimg_lw_coro_round_430_lvl1 = list_of_dictionaries('NRC_IMAGE', nrc_engimg_lw_detectors,
                                                         nrc_engimg_lw_round_430_subarrays,
                                                         nrc_engimg_lw_round_430_filterpupils,
                                                         #coronmasks=nrc_engimg_lw_round_430_mask,
                                                         detector1=True)
nrc_engimg_lw_coro_bar_lvl1 = list_of_dictionaries('NRC_IMAGE', nrc_engimg_lw_detectors,
                                                   nrc_engimg_lw_bar_subarrays,
                                                   nrc_engimg_lw_bar_filterpupils,
                                                   #coronmasks=nrc_engimg_lw_bar_mask,
                                                   detector1=True)

In [ ]:
nrc_engimg_fp_a3_lvl1 = list_of_dictionaries('NRC_IMAGE', nrc_engimg_fp_a3_detectors,
                                             nrc_engimg_fp_a3_subarrays,
                                             nrc_image_sw_filterpupil,
                                             detector1=True)
nrc_engimg_fp_b4_lvl1 = list_of_dictionaries('NRC_IMAGE', nrc_engimg_fp_b4_detectors,
                                             nrc_engimg_fp_b4_subarrays,
                                             nrc_image_sw_filterpupil,
                                             detector1=True)

In [ ]:
nrc_engimg_dhs_a_lvl1 = list_of_dictionaries('NRC_IMAGE', nrc_engimg_dhs_a_detectors,
                                             nrc_engimg_dhs_a_subarrays,
                                             nrc_engimg_dhs_filterpupil,
                                             detector1=True)
nrc_engimg_dhs_b_lvl1 = list_of_dictionaries('NRC_IMAGE', nrc_engimg_dhs_b_detectors,
                                             nrc_engimg_dhs_b_subarrays,
                                             nrc_engimg_dhs_filterpupil,
                                             detector1=True)

<a id='dark_combos'></a>
### Darks

NRC_DARK data are only processed through a specialized version of calwebb_detector1

In [ ]:
nrc_dark_sw_lvl1 = list_of_dictionaries(nrc_dark_exptype, nrc_dark_sw_dets, 
                                        nrc_dark_sw_subarrays, nrc_dark_sw_pupils,
                                        detector1=True)

In [ ]:
nrc_dark_lw_lvl1 = list_of_dictionaries(nrc_dark_exptype, nrc_dark_lw_dets, 
                                        nrc_dark_lw_subarrays, nrc_dark_lw_pupils,
                                        detector1=True)

In [ ]:
nrc_dark_p_sw_subs_lvl1 = list_of_dictionaries(nrc_dark_exptype, nrc_dark_sw_p_dets,
                                        nrc_dark_p_subarrays, nrc_dark_sw_pupils,
                                        detector1=True)

In [ ]:
nrc_dark_p_lw_subs_lvl1 = list_of_dictionaries(nrc_dark_exptype, nrc_dark_lw_p_dets,
                                        nrc_dark_p_subarrays, nrc_dark_lw_pupils,
                                        detector1=True)

In [ ]:
nrc_dark_grismts_sw_lvl1 = list_of_dictionaries(nrc_dark_exptype, nrc_dark_grismts_sw_dets,
                                                nrc_dark_grismts_sw_subarrays, nrc_dark_sw_pupils,
                                                detector1=True)

In [ ]:
nrc_dark_grismts_lw_lvl1 = list_of_dictionaries(nrc_dark_exptype, nrc_dark_grismts_lw_dets,
                                                nrc_dark_grismts_lw_subarrays, nrc_dark_lw_pupils,
                                                detector1=True)

<a id='ta_combos'></a>
### Target Acquisition

In [ ]:
nrc_coron_ta_sw_round_lvl1 = list_of_dictionaries(nrc_ta_exptype, nrc_coron_ta_round_detectors,
                                                  nrc_coron_ta_subarrays, nrc_coron_ta_sw_round_filterpupils,
                                                  coronmasks=nrc_coron_ta_round_mask, detector1=True)

In [ ]:
nrc_coron_ta_sw_bar_lvl1 = list_of_dictionaries(nrc_ta_exptype, nrc_coron_ta_bar_detectors,
                                                  nrc_coron_ta_subarrays, nrc_coron_ta_sw_bar_filterpupils,
                                                  coronmasks=nrc_coron_ta_bar_mask, detector1=True)

In [ ]:
nrc_coron_ta_lw_round_335_lvl1 = list_of_dictionaries(nrc_ta_exptype, nrc_coron_ta_lw_detectors,
                                                      nrc_coron_ta_subarrays,
                                                      nrc_coron_ta_lw_round_335_filterpupils,
                                                      coronmasks=nrc_coron_ta_round_335_mask,
                                                      detector1=True)

In [ ]:
nrc_coron_ta_lw_round_430_lvl1 = list_of_dictionaries(nrc_ta_exptype, nrc_coron_ta_lw_detectors,
                                                      nrc_coron_ta_subarrays,
                                                      nrc_coron_ta_lw_round_430_filterpupils,
                                                      coronmasks=nrc_coron_ta_round_430_mask,
                                                      detector1=True)

In [ ]:
nrc_coron_ta_lw_bar_lvl1 = list_of_dictionaries(nrc_ta_exptype, nrc_coron_ta_lw_detectors,
                                                nrc_coron_ta_subarrays,
                                                nrc_coron_ta_lw_bar_filterpupils,
                                                coronmasks=nrc_coron_ta_lw_bar_mask,
                                                detector1=True)

In [ ]:
nrc_tsimg_ta_lvl1 = list_of_dictionaries(nrc_ta_exptype, nrc_tsimg_ta_detectors, 
                                         nrc_tsimg_ta_subarrays, nrc_tsimg_ta_filterpupils,
                                         detector1=True)

In [ ]:
nrc_tsgrism_ta_lvl1 = list_of_dictionaries(nrc_ta_exptype, nrc_tsgrism_ta_detectors, 
                                           nrc_tsgrism_ta_subarrays, nrc_tsgrism_ta_filterpupils,
                                           detector1=True)

In [ ]:
nrc_coron_ta_sw_round_lvl2 = list_of_dictionaries(nrc_ta_exptype, nrc_coron_ta_round_detectors,
                                                  nrc_coron_ta_subarrays, nrc_coron_ta_sw_round_filterpupils,
                                                  coronmasks=nrc_coron_ta_round_mask, detector1=False)

In [ ]:
nrc_coron_ta_sw_bar_lvl2 = list_of_dictionaries(nrc_ta_exptype, nrc_coron_ta_bar_detectors,
                                                  nrc_coron_ta_subarrays, nrc_coron_ta_sw_bar_filterpupils,
                                                  coronmasks=nrc_coron_ta_bar_mask, detector1=False)

In [ ]:
nrc_coron_ta_lw_round_335_lvl2 = list_of_dictionaries(nrc_ta_exptype, nrc_coron_ta_lw_detectors,
                                                      nrc_coron_ta_subarrays,
                                                      nrc_coron_ta_lw_round_335_filterpupils,
                                                      coronmasks=nrc_coron_ta_round_335_mask,
                                                      detector1=False)

In [ ]:
nrc_coron_ta_lw_round_430_lvl2 = list_of_dictionaries(nrc_ta_exptype, nrc_coron_ta_lw_detectors,
                                                      nrc_coron_ta_subarrays,
                                                      nrc_coron_ta_lw_round_430_filterpupils,
                                                      coronmasks=nrc_coron_ta_round_430_mask,
                                                      detector1=False)

In [ ]:
nrc_coron_ta_lw_bar_lvl2 = list_of_dictionaries(nrc_ta_exptype, nrc_coron_ta_lw_detectors,
                                                nrc_coron_ta_subarrays,
                                                nrc_coron_ta_lw_bar_filterpupils,
                                                coronmasks=nrc_coron_ta_lw_bar_mask,
                                                detector1=False)

In [ ]:
nrc_tsimg_ta_lvl2 = list_of_dictionaries(nrc_ta_exptype, nrc_tsimg_ta_detectors, 
                                         nrc_tsimg_ta_subarrays, nrc_tsimg_ta_filterpupils,
                                         detector1=False)

In [ ]:
nrc_tsgrism_ta_lvl2 = list_of_dictionaries(nrc_ta_exptype, nrc_tsgrism_ta_detectors, 
                                           nrc_tsgrism_ta_subarrays, nrc_tsgrism_ta_filterpupils,
                                           detector1=False)

<a id='taconfirm_combos'></a>
### Target Acquisition Confirmation

In [ ]:
nrc_ta_conf_sw_round_lvl1 = list_of_dictionaries(nrc_ta_conf_exptype, nrc_ta_conf_sw_round_detectors,
                                                 nrc_ta_conf_subarrays, nrc_ta_conf_sw_round_filterpupils,
                                                 coronmasks=nrc_ta_conf_sw_round_mask, detector1=True)

In [ ]:
nrc_ta_conf_sw_bar_lvl1 = list_of_dictionaries(nrc_ta_conf_exptype, nrc_ta_conf_sw_bar_detectors,
                                                 nrc_ta_conf_subarrays, nrc_ta_conf_sw_bar_filterpupils,
                                                 coronmasks=nrc_ta_conf_sw_bar_mask, detector1=True)

In [ ]:
nrc_ta_conf_lw_round_335_lvl1 = list_of_dictionaries(nrc_ta_conf_exptype, nrc_ta_conf_lw_detectors,
                                                     nrc_ta_conf_subarrays,
                                                     nrc_ta_conf_lw_round_335_filterpupils, 
                                                     coronmasks=nrc_ta_conf_lw_round_335_mask,
                                                     detector1=True)

In [ ]:
nrc_ta_conf_lw_round_430_lvl1 = list_of_dictionaries(nrc_ta_conf_exptype, nrc_ta_conf_lw_detectors,
                                                     nrc_ta_conf_subarrays,
                                                     nrc_ta_conf_lw_round_430_filterpupils, 
                                                     coronmasks=nrc_ta_conf_lw_round_430_mask,
                                                     detector1=True)

In [ ]:
nrc_ta_conf_lw_bar_lvl1 = list_of_dictionaries(nrc_ta_conf_exptype, nrc_ta_conf_lw_detectors,
                                               nrc_ta_conf_subarrays,
                                               nrc_ta_conf_lw_bar_filterpupils, 
                                               coronmasks=nrc_ta_conf_lw_bar_mask,
                                               detector1=True)

In [ ]:
nrc_ta_conf_sw_round_lvl2 = list_of_dictionaries(nrc_ta_conf_exptype, nrc_ta_conf_sw_round_detectors,
                                                 nrc_ta_conf_subarrays, nrc_ta_conf_sw_round_filterpupils,
                                                 coronmasks=nrc_ta_conf_sw_round_mask, detector1=False)

In [ ]:
nrc_ta_conf_sw_bar_lvl2 = list_of_dictionaries(nrc_ta_conf_exptype, nrc_ta_conf_sw_bar_detectors,
                                                 nrc_ta_conf_subarrays, nrc_ta_conf_sw_bar_filterpupils,
                                                 coronmasks=nrc_ta_conf_sw_bar_mask, detector1=False)

In [ ]:
nrc_ta_conf_lw_round_335_lvl2 = list_of_dictionaries(nrc_ta_conf_exptype, nrc_ta_conf_lw_detectors,
                                                     nrc_ta_conf_subarrays,
                                                     nrc_ta_conf_lw_round_335_filterpupils, 
                                                     coronmasks=nrc_ta_conf_lw_round_335_mask,
                                                     detector1=False)

In [ ]:
nrc_ta_conf_lw_round_430_lvl2 = list_of_dictionaries(nrc_ta_conf_exptype, nrc_ta_conf_lw_detectors,
                                                     nrc_ta_conf_subarrays,
                                                     nrc_ta_conf_lw_round_430_filterpupils, 
                                                     coronmasks=nrc_ta_conf_lw_round_430_mask,
                                                     detector1=False)

In [ ]:
nrc_ta_conf_lw_bar_lvl2 = list_of_dictionaries(nrc_ta_conf_exptype, nrc_ta_conf_lw_detectors,
                                               nrc_ta_conf_subarrays,
                                               nrc_ta_conf_lw_bar_filterpupils, 
                                               coronmasks=nrc_ta_conf_lw_bar_mask,
                                               detector1=False)

<a id='focus_combos'></a>
### Focus

In [ ]:
nrc_focus_sw_lvl1 = list_of_dictionaries(nrc_focus_exptype, nrc_focus_sw_dets,
                                         nrc_focus_sw_subarrays,
                                         nrc_focus_sw_filterpupil, 
                                         detector1=True)
                                         
nrc_focus_sw_lvl2 = list_of_dictionaries(nrc_focus_exptype, nrc_focus_sw_dets,
                                         nrc_focus_sw_subarrays,
                                         nrc_focus_sw_filterpupil, 
                                         detector1=False)

In [ ]:
nrc_focus_lw_lvl1 = list_of_dictionaries(nrc_focus_exptype, nrc_focus_lw_dets,
                                         nrc_focus_lw_subarrays,
                                         nrc_focus_lw_filterpupil, 
                                         detector1=True)
                                         
nrc_focus_lw_lvl2 = list_of_dictionaries(nrc_focus_exptype, nrc_focus_lw_dets,
                                         nrc_focus_lw_subarrays,
                                         nrc_focus_lw_filterpupil, 
                                         detector1=False)

<a id='flat_combos'></a>
### Flat

In [ ]:
nrc_flat_sw_a_lvl1 = list_of_dictionaries(nrc_flat_exp_type, nrc_flat_sw_a_detectors,
                                          nrc_flat_sw_a_subarrays, nrc_flat_sw_a_filterpupils,
                                          detector1=True)

nrc_flat_sw_b_lvl1 = list_of_dictionaries(nrc_flat_exp_type, nrc_flat_sw_b_detectors,
                                          nrc_flat_sw_b_subarrays, nrc_flat_sw_b_filterpupils,
                                          detector1=True)

nrc_flat_lw_a_lvl1 = list_of_dictionaries(nrc_flat_exp_type, nrc_flat_lw_a_detectors,
                                          nrc_flat_lw_a_subarrays, nrc_flat_lw_a_filterpupils,
                                          detector1=True)

nrc_flat_lw_b_lvl1 = list_of_dictionaries(nrc_flat_exp_type, nrc_flat_lw_b_detectors,
                                          nrc_flat_lw_b_subarrays, nrc_flat_lw_b_filterpupils,
                                          detector1=True)

<a id='led_combos'></a>
### LED

In [ ]:
nrc_led_sw_lvl1 = list_of_dictionaries(nrc_led_exp_type, nrc_led_sw_detectors,
                                          nrc_led_sw_subarrays, nrc_led_sw_filterpupils,
                                          detector1=True)

nrc_led_lw_lvl1 = list_of_dictionaries(nrc_led_exp_type, nrc_led_lw_detectors,
                                          nrc_led_lw_subarrays, nrc_led_lw_filterpupils,
                                          detector1=True)

<a id='grism_combos'></a>
### GRISM

In [ ]:
nrc_grism_lw_lvl1 = list_of_dictionaries(nrc_engimg_lw_grism_exp_type, nrc_engimg_lw_grism_detectors,
                                          nrc_engimg_lw_grism_subarrays, nrc_engimg_lw_grism_filterpupils,
                                          detector1=True)
nrc_grism_lw_p_lvl1 = list_of_dictionaries(nrc_engimg_lw_grism_exp_type, nrc_engimg_lw_p_grism_detectors,
                                          nrc_engimg_lw_p_grism_subarrays, nrc_engimg_lw_grism_filterpupils,
                                          detector1=True)

---
<a id='run_queries'></a>
## Query CRDS

<a id='imaging_queries'></a>
### Imaging

Imaging mode data run through calwebb_detector1, calwebb_image2, and calwebb_image3, so we need to check for reference files associated with all three.

In [ ]:
missing_nrc_image_lvl1 = batch_query(nrc_image_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_image_lvl2 = batch_query(nrc_image_lvl2, level2a_imaging_reffiles)

In [ ]:
missing_nrc_image_lvl3 = batch_query(nrc_image_lvl2, level3_imaging_reffiles)

<a id='wfss_queries'></a>
### WFSS

In [ ]:
missing_nrc_wfss_lvl1 = batch_query(nrc_wfss_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_wfss_lvl2 = batch_query(nrc_wfss_lvl2, level2a_grism_reffiles)

In [ ]:
missing_nrc_wfss_lvl3 = batch_query(nrc_wfss_lvl2, level3_wfss_reffiles)

<a id='tsimage_queries'></a>
### Imaging Time Series

In [ ]:
missing_nrc_sw_tsimg_lvl1 = batch_query(nrc_tsimg_sw_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_lw_tsimg_lvl1 = batch_query(nrc_tsimg_lw_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_sw_tsgrism_lvl1 = batch_query(nrc_tsgrism_sw_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_sw_tsimg_lvl2 = batch_query(nrc_tsimg_sw_lvl2, level2a_imaging_reffiles)

In [ ]:
missing_nrc_lw_tsimg_lvl2 = batch_query(nrc_tsimg_lw_lvl2, level2a_imaging_reffiles)

In [ ]:
missing_nrc_sw_tsgrism_lvl2 = batch_query(nrc_tsgrism_sw_lvl2, level2a_imaging_reffiles)

In [ ]:
missing_nrc_sw_tsimg_lvl3 = batch_query(nrc_tsimg_sw_lvl2, level3_tso_reffiles)

In [ ]:
missing_nrc_lw_tsimg_lvl3 = batch_query(nrc_tsimg_lw_lvl2, level3_tso_reffiles)

In [ ]:
missing_nrc_sw_tsgrism_lvl3 = batch_query(nrc_tsgrism_sw_lvl2, level3_tso_reffiles)

<a id='tsgrism_queries'></a>
### Grism Time Series

In [ ]:
missing_nrc_lw_tsgrism_lvl1 = batch_query(nrc_tsgrism_lw_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_lw_tsgrism_lvl2 = batch_query(nrc_tsgrism_lw_lvl2, level2a_grism_reffiles)

In [ ]:
missing_nrc_lw_tsgrism_lvl3 = batch_query(nrc_tsgrism_lw_lvl2, level3_tso_reffiles)

<a id='coron_queries'></a>
### Coronagraphy

In [ ]:
missing_nrc_sw_coron_round_lvl1 = batch_query(nrc_coron_sw_round_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_sw_coron_bar_lvl1 = batch_query(nrc_coron_sw_bar_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_lw_coron_round_335_lvl1 = batch_query(nrc_coron_lw_round_335_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_lw_coron_round_430_lvl1 = batch_query(nrc_coron_lw_round_430_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_lw_coron_bar_lvl1 = batch_query(nrc_coron_lw_bar_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_sw_coron_round_lvl2 = batch_query(nrc_coron_sw_round_lvl2, level2a_imaging_reffiles)

In [ ]:
missing_nrc_sw_coron_bar_lvl2 = batch_query(nrc_coron_sw_bar_lvl2, level2a_imaging_reffiles)

In [ ]:
missing_nrc_lw_coron_round_335_lvl2 = batch_query(nrc_coron_lw_round_335_lvl2, level2a_imaging_reffiles)

In [ ]:
missing_nrc_lw_coron_round_430_lvl2 = batch_query(nrc_coron_lw_round_430_lvl2, level2a_imaging_reffiles)

In [ ]:
missing_nrc_lw_coron_bar_lvl2 = batch_query(nrc_coron_lw_bar_lvl2, level2a_imaging_reffiles)

In [ ]:
missing_nrc_sw_coron_round_lvl3 = batch_query(nrc_coron_sw_round_lvl2, level3_coron_reffiles)

In [ ]:
missing_nrc_sw_coron_bar_lvl3 = batch_query(nrc_coron_sw_bar_lvl2, level3_coron_reffiles)

In [ ]:
missing_nrc_lw_coron_round_335_lvl3 = batch_query(nrc_coron_lw_round_335_lvl2, level3_coron_reffiles)

In [ ]:
missing_nrc_lw_coron_round_430_lvl3 = batch_query(nrc_coron_lw_round_430_lvl2, level3_coron_reffiles)

In [ ]:
missing_nrc_lw_coron_bar_lvl3 = batch_query(nrc_coron_lw_bar_lvl2, level3_coron_reffiles)

<a id='engimg_queries'></a>
### Engineering Imaging

In [ ]:
missing_nrc_engimg_sw_lvl1 = batch_query(nrc_engimg_sw_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_engimg_lw_lvl1 = batch_query(nrc_engimg_lw_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_engimg_sw_coro_round_lvl1 = batch_query(nrc_engimg_sw_coro_round_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_engimg_sw_coro_bar_lvl1 = batch_query(nrc_engimg_sw_coro_bar_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_engimg_lw_coro_round_335_lvl1 = batch_query(nrc_engimg_lw_coro_round_335_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_engimg_lw_coro_round_430_lvl1 = batch_query(nrc_engimg_lw_coro_round_430_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_engimg_lw_coro_bar_lvl1 = batch_query(nrc_engimg_lw_coro_bar_lvl1, level1b_reffiles)

How important are the missing reference files above? Will anyone be using engineering imaging with the coronagraphic masks in place?

In [ ]:
missing_nrc_engimg_fp_a3_lvl1 = batch_query(nrc_engimg_fp_a3_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_engimg_fp_b4_lvl1 = batch_query(nrc_engimg_fp_b4_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_engimg_dhs_a_lvl1 = batch_query(nrc_engimg_dhs_a_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_engimg_dhs_b_lvl1 = batch_query(nrc_engimg_dhs_b_lvl1, level1b_reffiles)

<a id='dark_queries'></a>
### Darks

In [ ]:
missing_nrc_dark_sw_lvl1 = batch_query(nrc_dark_sw_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_dark_lw_lvl1 = batch_query(nrc_dark_lw_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_dark_p_sw_lvl1 = batch_query(nrc_dark_p_sw_subs_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_dark_p_lw_lvl1 = batch_query(nrc_dark_p_lw_subs_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_dark_grismts_sw_lvl1 = batch_query(nrc_dark_grismts_sw_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_dark_grismts_lw_lvl1 = batch_query(nrc_dark_grismts_lw_lvl1, level1b_reffiles)

There must be more modes that need to be covered here?? Or since we're not subtracting a dark from these, all the reference files we are looking for are full-frame?

<a id='ta_queries'></a>
### Target Acquisition

In [ ]:
missing_nrc_ta_coron_sw_round_lvl1 = batch_query(nrc_coron_ta_sw_round_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_ta_coron_sw_bar_lvl1 = batch_query(nrc_coron_ta_sw_bar_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_ta_coron_lw_round_335_lvl1 = batch_query(nrc_coron_ta_lw_round_335_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_ta_coron_lw_round_430_lvl1 = batch_query(nrc_coron_ta_lw_round_430_lvl1, level1b_reffiles)

In [ ]:
missing_nrc_ta_coron_lw_bar_lvl1 = batch_query(nrc_coron_ta_lw_bar_lvl1, level1b_reffiles)

TA files will eventually be processed through level 2b. 

In [ ]:
missing_nrc_ta_coron_sw_round_lvl2 = batch_query(nrc_coron_ta_sw_round_lvl2, level2a_imaging_reffiles)

In [ ]:
missing_nrc_ta_coron_sw_bar_lvl2 = batch_query(nrc_coron_ta_sw_bar_lvl2, level2a_imaging_reffiles)

In [ ]:
missing_nrc_ta_coron_lw_round_335_lvl2 = batch_query(nrc_coron_ta_lw_round_335_lvl2, level2a_imaging_reffiles)

In [ ]:
missing_nrc_ta_coron_lw_round_430_lvl2 = batch_query(nrc_coron_ta_lw_round_430_lvl2, level2a_imaging_reffiles)

In [ ]:
missing_nrc_ta_coron_lw_bar_lvl2 = batch_query(nrc_coron_ta_lw_bar_lvl2, level2a_imaging_reffiles)

<a id='taconfirm_queries'></a>
### Target Acquisition Confirmation

In [ ]:
missing_ta_conf_sw_round_lvl1 = batch_query(nrc_ta_conf_sw_round_lvl1, level1b_reffiles)

In [ ]:
missing_ta_conf_sw_round_lvl1

In [ ]:
missing_ta_conf_sw_bar_lvl1 = batch_query(nrc_ta_conf_sw_bar_lvl1, level1b_reffiles)

In [ ]:
missing_ta_conf_sw_bar_lvl1

In [ ]:
missing_ta_conf_lw_round_335_lvl1 = batch_query(nrc_ta_conf_lw_round_335_lvl1, level1b_reffiles)

In [ ]:
missing_ta_conf_lw_round_335_lvl1

In [ ]:
missing_ta_conf_lw_round_430_lvl1 = batch_query(nrc_ta_conf_lw_round_430_lvl1, level1b_reffiles)

In [ ]:
missing_ta_conf_lw_round_430_lvl1

In [ ]:
missing_ta_conf_lw_bar_lvl1 = batch_query(nrc_ta_conf_lw_bar_lvl1, level1b_reffiles)

In [ ]:
missing_ta_conf_lw_bar_lvl1

TA confirmation files will eventually also be processed through level 2b??

In [ ]:
missing_ta_conf_sw_round_lvl2 = batch_query(nrc_ta_conf_sw_round_lvl2, level2a_imaging_reffiles)

In [ ]:
missing_ta_conf_sw_bar_lvl2 = batch_query(nrc_ta_conf_sw_bar_lvl2, level2a_imaging_reffiles)

In [ ]:
missing_ta_conf_lw_round_335_lvl2 = batch_query(nrc_ta_conf_lw_round_335_lvl2, level2a_imaging_reffiles)

In [ ]:
missing_ta_conf_lw_round_430_lvl2 = batch_query(nrc_ta_conf_lw_round_430_lvl2, level2a_imaging_reffiles)

In [ ]:
missing_ta_conf_lw_bar_lvl2 = batch_query(nrc_ta_conf_lw_bar_lvl2, level2a_imaging_reffiles)

<a id='focus_queries'></a>
### Focus

In [ ]:
missing_focus_sw_lvl1 = batch_query(nrc_focus_sw_lvl1, level1b_reffiles)

In [ ]:
missing_focus_sw_lvl2 = batch_query(nrc_focus_sw_lvl2, level2a_imaging_reffiles)

In [ ]:
missing_focus_lw_lvl1 = batch_query(nrc_focus_lw_lvl1, level1b_reffiles)

In [ ]:
missing_focus_lw_lvl2 = batch_query(nrc_focus_lw_lvl2, level2a_imaging_reffiles)

<a id='flat_queries'></a>
### Flat

In [ ]:
missing_flat_sw_a_lvl1 = batch_query(nrc_flat_sw_a_lvl1, level1b_reffiles)

In [ ]:
missing_flat_sw_b_lvl1 = batch_query(nrc_flat_sw_b_lvl1, level1b_reffiles)

In [ ]:
missing_flat_lw_a_lvl1 = batch_query(nrc_flat_lw_a_lvl1, level1b_reffiles)

In [ ]:
missing_flat_lw_b_lvl1 = batch_query(nrc_flat_lw_b_lvl1, level1b_reffiles)

<a id='grism_queries'></a>
### GRISM

In [ ]:
missing_grism_lw_lvl1 = batch_query(nrc_grism_lw_lvl1, level1b_reffiles)

In [ ]:
missing_grism_p_lw_lvl1 = batch_query(nrc_grism_lw_p_lvl1, level1b_reffiles)

---
<a id='final_summary'></a>
## Final Summary

### Imaging

In [ ]:
missing_nrc_image_lvl1

In [ ]:
missing_nrc_image_lvl2

In [ ]:
missing_nrc_image_lvl3

### WFSS

In [ ]:
missing_nrc_wfss_lvl1

In [ ]:
missing_nrc_wfss_lvl2

In [ ]:
missing_nrc_wfss_lvl3

### Imaging Time Series

In [ ]:
missing_nrc_sw_tsimg_lvl1

In [ ]:
missing_nrc_lw_tsimg_lvl1

In [ ]:
missing_nrc_sw_tsgrism_lvl1

In [ ]:
missing_nrc_sw_tsimg_lvl2

In [ ]:
missing_nrc_lw_tsimg_lvl2

In [ ]:
missing_nrc_sw_tsgrism_lvl2

In [ ]:
missing_nrc_sw_tsimg_lvl3

In [ ]:
missing_nrc_lw_tsimg_lvl3

In [ ]:
missing_nrc_sw_tsgrism_lvl3

### Grism Time Series

In [ ]:
missing_nrc_lw_tsgrism_lvl1

In [ ]:
missing_nrc_lw_tsgrism_lvl2

In [ ]:
missing_nrc_lw_tsgrism_lvl3

### Coronagraphy

In [ ]:
missing_nrc_sw_coron_round_lvl1

In [ ]:
missing_nrc_sw_coron_bar_lvl1

In [ ]:
missing_nrc_lw_coron_round_335_lvl1

In [ ]:
missing_nrc_lw_coron_round_430_lvl1

In [ ]:
missing_nrc_lw_coron_bar_lvl1

In [ ]:
missing_nrc_sw_coron_round_lvl2

In [ ]:
missing_nrc_sw_coron_bar_lvl2

In [ ]:
missing_nrc_lw_coron_round_335_lvl2

In [ ]:
missing_nrc_lw_coron_round_430_lvl2

In [ ]:
missing_nrc_lw_coron_bar_lvl2

In [ ]:
missing_nrc_sw_coron_round_lvl3

In [ ]:
missing_nrc_sw_coron_bar_lvl3

In [ ]:
missing_nrc_lw_coron_round_335_lvl3

In [ ]:
missing_nrc_lw_coron_round_430_lvl3

In [ ]:
missing_nrc_lw_coron_bar_lvl3

### Engineering Imaging

In [ ]:
missing_nrc_engimg_sw_lvl1

In [ ]:
missing_nrc_engimg_lw_lvl1

In [ ]:
missing_nrc_engimg_sw_coro_round_lvl1

In [ ]:
missing_nrc_engimg_sw_coro_bar_lvl1

In [ ]:
missing_nrc_engimg_lw_coro_round_335_lvl1

In [ ]:
missing_nrc_engimg_lw_coro_round_430_lvl1

In [ ]:
missing_nrc_engimg_lw_coro_bar_lvl1

In [ ]:
missing_nrc_engimg_fp_a3_lvl1

In [ ]:
missing_nrc_engimg_fp_b4_lvl1

In [ ]:
missing_nrc_engimg_dhs_a_lvl1

In [ ]:
missing_nrc_engimg_dhs_b_lvl1

<a id='dark_queries'></a>
### Darks

In [ ]:
missing_nrc_dark_sw_lvl1

In [ ]:
missing_nrc_dark_lw_lvl1

In [ ]:
missing_nrc_dark_p_sw_lvl1

In [ ]:
missing_nrc_dark_p_lw_lvl1

In [ ]:
missing_nrc_dark_grismts_sw_lvl1

In [ ]:
missing_nrc_dark_grismts_lw_lvl1

<a id='ta_queries'></a>
### Target Acquisition

In [ ]:
missing_nrc_ta_coron_sw_round_lvl1

In [ ]:
missing_nrc_ta_coron_sw_bar_lvl1

In [ ]:
missing_nrc_ta_coron_lw_round_335_lvl1

In [ ]:
missing_nrc_ta_coron_lw_round_430_lvl1

In [ ]:
missing_nrc_ta_coron_lw_bar_lvl1

TA files will eventually be processed through level 2b. 

In [ ]:
missing_nrc_ta_coron_sw_round_lvl2

In [ ]:
missing_nrc_ta_coron_sw_bar_lvl2

In [ ]:
missing_nrc_ta_coron_lw_round_335_lvl2

In [ ]:
missing_nrc_ta_coron_lw_round_430_lvl2

In [ ]:
missing_nrc_ta_coron_lw_bar_lvl2

<a id='taconfirm_queries'></a>
### Target Acquisition Confirmation

In [ ]:
missing_ta_conf_sw_round_lvl1

In [ ]:
missing_ta_conf_sw_bar_lvl1

In [ ]:
missing_ta_conf_lw_round_335_lvl1

In [ ]:
missing_ta_conf_lw_round_430_lvl1

In [ ]:
missing_ta_conf_lw_bar_lvl1

In [ ]:
missing_ta_conf_sw_round_lvl2

In [ ]:
missing_ta_conf_sw_bar_lvl2

In [ ]:
missing_ta_conf_lw_round_335_lvl2

In [ ]:
missing_ta_conf_lw_round_430_lvl2

In [ ]:
missing_ta_conf_lw_bar_lvl2

<a id='focus_queries'></a>
### Focus

In [ ]:
missing_focus_sw_lvl1

In [ ]:
missing_focus_sw_lvl2

In [ ]:
missing_focus_lw_lvl1

In [ ]:
missing_focus_lw_lvl2

<a id='flat_queries'></a>
### Flat

In [ ]:
missing_flat_sw_a_lvl1

In [ ]:
missing_flat_sw_b_lvl1

In [ ]:
missing_flat_lw_a_lvl1

In [ ]:
missing_flat_lw_b_lvl1

<a id='grism_queries'></a>
### GRISM

In [ ]:
missing_grism_lw_lvl1

In [ ]:
missing_grism_p_lw_lvl1